In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas
from sklearn.model_selection import train_test_split

2.1.0


In [2]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [3]:
ds = pandas.read_csv("./dataset.csv")
X = ds.iloc[:, :-1].values
y = ds.iloc[:, -1].values

In [20]:

train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size=0.1, shuffle=True)

train_data = torch.from_numpy(train_data).float()
test_data = torch.from_numpy(test_data).float()
train_labels = torch.from_numpy(train_labels).long()
test_labels = torch.from_numpy(test_labels).long()

train_dataset = CustomDataset(train_data, train_labels)
test_dataset = CustomDataset(test_data, test_labels)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [21]:
class FeedForward(nn.Module):
    def __init__(self):
        super(FeedForward, self).__init__()
        self.func1 = nn.Linear(2, 32) 
        self.relufunc = nn.ReLU()
        self.func2 = nn.Linear(32, 4)
    
    def forward(self, x):
        x = self.func1(x)
        x = self.relufunc(x)
        x = self.func2(x)
      
        x = F.softmax(x, dim=0)
        return x


In [23]:
num_epochs = 100
learning_rate = 0.001

model = FeedForward()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        outputs = model(inputs)
        targets -= 1
        loss = criterion(outputs, targets.long())
        
        _, predicted = torch.max(outputs.data, 1)
        total = targets.size(0)
        correct = (predicted == targets).sum().item()
        accuracy = correct / total

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'e [{epoch+1}/{num_epochs}], Loss: {loss.item()}, Accuracy: {accuracy*100}%')



e [1/100], Loss: 1.3492270708084106, Accuracy: 87.5%
e [2/100], Loss: 1.3208187818527222, Accuracy: 93.75%
e [3/100], Loss: 1.2989379167556763, Accuracy: 93.75%
e [4/100], Loss: 1.2690439224243164, Accuracy: 93.75%
e [5/100], Loss: 1.2559317350387573, Accuracy: 93.75%
e [6/100], Loss: 1.2808897495269775, Accuracy: 81.25%
e [7/100], Loss: 1.277245283126831, Accuracy: 81.25%
e [8/100], Loss: 1.219722867012024, Accuracy: 93.75%
e [9/100], Loss: 1.2129298448562622, Accuracy: 100.0%
e [10/100], Loss: 1.2316428422927856, Accuracy: 87.5%
e [11/100], Loss: 1.2387887239456177, Accuracy: 75.0%
e [12/100], Loss: 1.2506983280181885, Accuracy: 68.75%
e [13/100], Loss: 1.2527567148208618, Accuracy: 81.25%
e [14/100], Loss: 1.2377221584320068, Accuracy: 87.5%
e [15/100], Loss: 1.2194663286209106, Accuracy: 87.5%
e [16/100], Loss: 1.2482388019561768, Accuracy: 75.0%
e [17/100], Loss: 1.2218037843704224, Accuracy: 87.5%
e [18/100], Loss: 1.222231149673462, Accuracy: 93.75%
e [19/100], Loss: 1.213598847

In [24]:
model.eval()  
with torch.no_grad():
    correct = 0
    total = 0
    for i, (inputs, targets) in enumerate(test_loader):
        outputs = model(inputs)
        targets -= 1
        loss = criterion(outputs, targets)
        
     
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        accuracy = correct / total

print(f'Loss: {loss.item()}, Accuracy: {accuracy*100}%')

Loss: 1.2124232053756714, Accuracy: 94.0%
